In [ ]:
from plantcv import plantcv as pcv
import videometer.hips as hips
import numpy as np
import argparse
import glob
from random import shuffle
import plantcv.plantcv as pcv
from skimage import img_as_ubyte

In [ ]:
# Set global debug behavior to None (default), "print" (to file), 
# or "plot" (Jupyter Notebooks or X11)
pcv.params.debug = "plot"
pcv.params.text_size = 5
pcv.params.text_thickness = 5

In [ ]:
def main():
    # define directories
    indir = 'C:/input_directory'
    outdir = 'C:/output_directory'
    plot = "TRUE"

    # go through files in directory randomly
    files = glob.glob(f"{indir}/*.raw")
    shuffle(files)

    for f in files:
        print(f)
        ## read image, create full seed mask
        img = pcv.readimage(filename=f, mode="envi")
        ## convert pseudo rgb image to a gray HSV image, select channel 'h'
        c_img = pcv.rgb2gray_hsv(rgb_img=img.pseudo_rgb, channel='h')
        ## threshold dark objects (seeds)
        c_thresh = pcv.threshold.otsu(gray_img=c_img, object_type='dark')
        ## fill holes to remove spaces within seeds
        c_fill = pcv.fill_holes(bin_img=c_thresh)
        ## define region of interest
        roi = pcv.roi.rectangle(img=img.pseudo_rgb, x=277, y=100, h=2700, w=3750)
        ## fill objects less than 20000 pixels
        a_fill_image = pcv.fill(bin_img=c_fill, size=20000)
        ## filter objects within ROI, keep seeds
        flt_mask = pcv.roi.quick_filter(mask=a_fill_image, roi=roi)
        ## define grid of ROIs
        ## if this step fails, skip this plate and move on the the next
        try:
            rois = pcv.roi.auto_grid(mask=flt_mask, nrows=4, ncols=6, radius=250, img=img.pseudo_rgb)
        except RuntimeError as err:
            print(f"{f} failed at auto_grid(); skipping")
            continue
        except ValueError as err:
            print(f"{f} failed at auto_grid(); skipping")
            continue
        ## create labeled mask for data extraction
        labeled_mask, num = pcv.create_labels(mask=flt_mask, rois=rois)
    
        # make first wing mask
        ## select image of wavelength 940nm
        r940 = img_as_ubyte(img.array_data[: , :, -2] / 255)
        ## apply seed mask to multispectral image
        masked_r940 = pcv.apply_mask(img=r940, mask=flt_mask, mask_color="black")
        ## threshold out light objects (wings)
        wing_mask = pcv.threshold.binary(gray_img=masked_r940, threshold=40, object_type="light")
        ## erode wing
        wing_err = pcv.erode(gray_img=wing_mask, ksize=4, i=1)
        ## dilate wing
        wing_mask_dil = pcv.dilate(gray_img=wing_err, ksize=4, i=2)
        ## fill small objects to eliminate noise
        wing_mask_clean = pcv.fill(bin_img=wing_mask_dil, size=2500)
    
        # make kernel mask
        ## subtract first wing mask from seed mask to leave kernels
        kernel_mask = flt_mask - wing_mask
        ## erode kernel mask to remove remaining white space in mask
        kernel_err = pcv.erode(gray_img=kernel_mask, ksize=4, i=5)
        ## fill mask to eliminate noise
        kernel_mask_clean = pcv.fill(bin_img=kernel_err, size=2900)
        ## dilate kernels
        kernel_mask_dil = pcv.dilate(gray_img=kernel_mask_clean, ksize=4, i=4)
        ## fill holes in kernels
        kernel_mask_fill = pcv.fill_holes(bin_img=kernel_mask_dil)
        ## create labeled mask for data extraction
        kernel_labeled_mask, num = pcv.create_labels(mask=kernel_mask_fill, rois=rois)
        
        # make final wing mask
        ## subtract kernel mask from full mask
        wing_mask_final = flt_mask - kernel_mask_fill
        ## create labeled mask for data extraction
        wing_labeled_mask, num = pcv.create_labels(mask=wing_mask_final, rois=rois)

        # extract shape, spectral, and HSV data per seed, wing, and kernel from labeled masks
        ## whole seed data extraction
        seed_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=labeled_mask, n_labels=num, label="seed")
        seed_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=labeled_mask, n_labels=num, label="seed")
        seed_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=labeled_mask, n_labels=num, label='seed')
    
        ## wing data extraction
        wing_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=wing_labeled_mask, n_labels=num, label="wing")
        wing_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=wing_labeled_mask, n_labels=num, label="wing")
        wing_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=wing_labeled_mask, n_labels=num, label='wing')
    
        ## kernel data extraction
        kernel_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=kernel_labeled_mask, n_labels=num, label="kernel")
        kernel_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=kernel_labeled_mask, n_labels=num, label="kernel")
        kernel_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=kernel_labeled_mask, n_labels=num, label='kernel')

        # save masks
        if plot == "TRUE":
            outfile = f.removesuffix('.raw').removeprefix(indir)
            ## save seed mask for each plate
            x = pcv.visualize.pseudocolor(c_fill, mask=labeled_mask, background='black')
            x.savefig(f'{outdir}/{outfile} Seed.png')
            ## save wing mask for each plate
            y = pcv.visualize.pseudocolor(c_fill, mask=wing_labeled_mask, background='black')
            y.savefig(f'{outdir}/{outfile} Wing.png')
            ## save kernel mask for each plate
            z = pcv.visualize.pseudocolor(c_fill, mask=kernel_labeled_mask, background='black')
            z.savefig(f'{outdir}/{outfile} Kernel.png')
    
        # save outputs
        print(outdir)
        outfile = f.removesuffix('.raw').removeprefix(indir)
        pcv.outputs.save_results(filename=f"{outdir}/{outfile}.csv", outformat="csv")

    return 1
if __name__ == '__main__':
    main()